In [1]:
import numpy as np

In [11]:

def generateRomPlots(mechanismKinematics, actuatorParams, n, title, color):
    thetaPitchVec = np.linspace(-80,80, n)
    thetaRollVec = np.linspace(-25,25, n)

    outsideActuatorLengthsMatrix = np.zeros((10,10))
    insideActuatorLengthsMatrix = np.zeros((10,10))

    for i in range(len(thetaPitchVec)):
        for j in range(len(thetaRollVec)):
            thetaPitch = thetaPitchVec[i]
            thetaRoll = thetaRollVec[j]
            #actuatorLengths = 
            print(thetaPitch)
            print(thetaRoll)

In [12]:
generateRomPlots(1, 1, 10, "name", "color")

-80.0
-25.0
-80.0
-19.444444444444443
-80.0
-13.88888888888889
-80.0
-8.333333333333336
-80.0
-2.7777777777777786
-80.0
2.7777777777777786
-80.0
8.333333333333329
-80.0
13.888888888888886
-80.0
19.444444444444443
-80.0
25.0
-62.22222222222222
-25.0
-62.22222222222222
-19.444444444444443
-62.22222222222222
-13.88888888888889
-62.22222222222222
-8.333333333333336
-62.22222222222222
-2.7777777777777786
-62.22222222222222
2.7777777777777786
-62.22222222222222
8.333333333333329
-62.22222222222222
13.888888888888886
-62.22222222222222
19.444444444444443
-62.22222222222222
25.0
-44.44444444444444
-25.0
-44.44444444444444
-19.444444444444443
-44.44444444444444
-13.88888888888889
-44.44444444444444
-8.333333333333336
-44.44444444444444
-2.7777777777777786
-44.44444444444444
2.7777777777777786
-44.44444444444444
8.333333333333329
-44.44444444444444
13.888888888888886
-44.44444444444444
19.444444444444443
-44.44444444444444
25.0
-26.666666666666664
-25.0
-26.666666666666664
-19.444444444444443
-2

In [ ]:
def computeActuatorLengthsExtended(mechanismKinematics, thetaPitch, thetaRoll):
    x = mechanismKinematics.leftOutsideAnkleFootAttachment[0]
    y_outside = mechanismKinematics.leftOutsideAnkleFootAttachment[1]
    y_inside = mechanismKinematics.leftInsideAnkleFootAttachment[1]
    z = mechanismKinematics.leftOutsideAnkleFootAttachment[2]

    #put angles in same direction as onshape
    thetaPitch = -thetaPitch

    axisOffset = mechanismKinematics.RollPitchAxisOffeset
    pitchAxis = np.add(axisOffset, np.array([x, 0, 0]))  
    rollAxis = np.array([x, 0, z])
    insideActuator = np.array([x, y_outside, z])
    outsideAcuator = np.array([x, y_inside, z])

    #Rotate axisOffset & pitchAxis around y-axis by thetaPitch
    v = np.array([axisOffset, pitchAxis, rollAxis, insideActuator, outsideAcuator])

    #Y-Axis Unit Vector
    yAxis = np.array([0, 1, 0])

    